In [ ]:
%matplotlib notebook

import matplotlib as mpl
from MalardClient.MalardClient import MalardClient
from MalardClient.DataSet import DataSet
from MalardClient.BoundingBox import BoundingBox
import datetime
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import os
import numpy as np
import pandas as  pd
import json
import math
from pandas.io.json import json_normalize
from matplotlib.colors import LinearSegmentedColormap

from datetime import datetime

In [ ]:

#ds = DataSet('cryotempo', 'GRIS_BaselineC_Q2', 'greenland')
ds = DataSet('cryotempo', 'AntarcticaV2b', 'antarctic')

runName='test'
size = 100000

client = MalardClient()

bbox = client.boundingBox( ds )

print(bbox)

In [ ]:
#stats = query.getRunStatistics(parentDsName, runName)
#stats = json.loads(stats)
#dfStats = json_normalize(stats)
#dfStats.head()

bbox = BoundingBox( bbox.minX, bbox.maxX, bbox.minY, bbox.maxY,  datetime(2017,1,1,0,0,0), datetime(2017,12,31,0,0,0) )


gcs = client.gridCells(ds, bbox)



In [ ]:
# background layers (must be same projection)
#rivers = gp.read_file("/data/puma1/scratch/mtngla/background-shapefiles/rivers_reprojected.shp")
countries = gp.read_file("/data/puma1/scratch/cryotempo/masks/ice.shp")

In [ ]:
# OPTION 1 (display as points)
# convert center points to geo points
crs = client.getProjection(ds).proj4

def toGeoData(data):
    # Convert gridcells to point geometries in the middle of the cell
    geometry = [Point(xy) for xy in zip(data['gridCell.minX']+(size/2), data['gridCell.minY']+(size/2))]
    gData = gp.GeoDataFrame(data, crs=crs, geometry=geometry)
    return gData

dfPoints = toGeoData(dfStats.copy())

In [ ]:
# OPTION 2 (display as squares)
# convert to polygons
crs = client.getProjection(ds).proj4

def toGeoData(data):
    #data.drop(['geometry'], axis=1)
    geometry = []
    totals = []
    for gc in data:
        geometry.append(Polygon([(gc.minX,gc.minY), (gc.minX,gc.maxY), (gc.maxX,gc.maxY), (gc.maxX,gc.minY), (gc.minX,gc.minY)]))
        totals.append(gc.numberOfPoints)
        
        
    df = pd.DataFrame()
    df['TotalPoints'] = totals
    
    gData = gp.GeoDataFrame(df, crs=crs, geometry=geometry)
    
      
    return gData

dfPolys = toGeoData(gcs)

In [ ]:
# OPTION 1: define point sizes
dfPoints.loc[(dfPoints['statistics.GlacierMaskArea']<10.0), 'statistics.GlacierMaskArea.pointsize'] = 5
dfPoints.loc[(dfPoints['statistics.GlacierMaskArea']>=10.0) & (dfPoints['statistics.GlacierMaskArea']<100.0), 'statistics.GlacierMaskArea.pointsize'] = 20
dfPoints.loc[(dfPoints['statistics.GlacierMaskArea']>=100.0) & (dfPoints['statistics.GlacierMaskArea']<500.0), 'statistics.GlacierMaskArea.pointsize'] = 50
dfPoints.loc[(dfPoints['statistics.GlacierMaskArea']>=500.0) & (dfPoints['statistics.GlacierMaskArea']<1000.0), 'statistics.GlacierMaskArea.pointsize'] = 80
dfPoints.loc[(dfPoints['statistics.GlacierMaskArea']>=1000.0) & (dfPoints['statistics.GlacierMaskArea']<2000.0), 'statistics.GlacierMaskArea.pointsize'] = 125
dfPoints.loc[(dfPoints['statistics.GlacierMaskArea']>2000.0), 'statistics.GlacierMaskArea.pointsize'] = 170

In [ ]:
# OPTION 1: plot
%matplotlib notebook

# plot figure
fig, ax = plt.subplots(figsize=(11,5))

# plot background
countries.plot(ax=ax, edgecolor='grey', color='gainsboro', linewidth=0.5)
rivers.plot(ax=ax, edgecolor='blue', linewidth=0.2)

# marker size
markersize = dfPoints['statistics.GlacierMaskArea.pointsize']

# min and max of colour scheme (change this!)
vmin=0
vmax=20000

# color scheme
cm = 'viridis'

# plot data
dfPoints.plot(ax=ax, column='statistics.result_total', markersize=markersize, vmin=vmin, \
                     vmax=vmax, cmap=cm)

# marker size legend
labels = ["<10", "10-100", "100-500", "500-1000", "1000-2000", ">2000"]
sizes = [5, 20, 50, 80, 125, 170]
markers=[]
for size in sizes:
    markers.append(plt.scatter([],[], s=size, label=size, color='white', edgecolor='black', linewidth=0.5))
plt.legend(handles=markers)
leg = plt.legend(markers, labels, ncol=1, frameon=True, fontsize=7,handlelength=1, loc = 3, borderpad = 1.3,\
handletextpad=1, title='Glacier area [km2]', scatterpoints = 1, labelspacing=1.5)

# add colorbar with label
bar=fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax), cmap=cm))
bar.set_label('Total number of points')

# add plot title
plt.title('Title blah')

In [ ]:
# OPTION 2: plot
%matplotlib notebook

# plot figure
fig, ax = plt.subplots(figsize=(11,8))

# plot background
#countries.plot(ax=ax, edgecolor='grey', color='gainsboro', linewidth=0.5)
#rivers.plot(ax=ax, edgecolor='blue', linewidth=0.2)

# marker size
markersize = 1 #dfPoints['statistics.GlacierMaskArea.pointsize']

# min and max of colour scheme (change this!)

totals = [ gc.numberOfPoints for gc in gcs ]

diff = max(totals) - min(totals)
vmin= min(totals) #+ diff*0.25
vmax= max(totals)# - diff*0.25

#vmin = 6500000
#vmax = 7000000

# color scheme
cm = 'plasma'

# plot data
#dfPolys.plot(ax=ax, column='TotalPoints', vmin=vmin, \
#                     vmax=vmax, cmap=cm, alpha=0.8)
dfPolys.plot(ax=ax, column='TotalPoints',color='lightblue',edgecolor='dimgray', alpha=0.5)

# add colorbar with label
#bar=fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax), cmap=cm))
#bar.set_label('Total number of points')

# add plot title
plt.title('Antarctic area coverage with 100km tiles')
ax.set_xlabel("X Coordinate (m)")
ax.set_ylabel("Y Coordinate (m)")


In [ ]:
print(gcs[100].numberOfPoints)

In [ ]:
print(dfPolys.TotalPoints[100])

In [ ]:
fig.savefig('antarctica_area_coverage_2017.png')